###What's been done in this notebook:

1. Stood up Spark cluster

2. uploaded listings.csv as a table into the default database

3. Cleaned dirty data since sample was locked down.

4. Replicated Siavash's notebook he sent me for timings

5. Tested how long it would take to calculate a root mean squared error on 82029 records.

###Functions and timing:

1. Count() - 14.8s

2. Replace nulls with '0', regex_replace special characters, change 6 columns to float data type - 1.13s

3. Show histogram of all prices - 17.31s

4. Plot a GroupBy and Count of 'Neighborhood_cleansed' column - 14.80s

5. Calculate average price and median price - 27s

6. Calculate RMSE for both Avg Price and Median columns - 28.5s

In [2]:
%%time

# loading table into spark format
listings = spark.table('listings_csv')

# count total number of records
recordCount = listings.count()

print("Total number of records: " + str(recordCount))

Total number of records: 82029
CPU times: user 85.3 ms, sys: 26.2 ms, total: 111 ms
Wall time: 21.3 s

This isn't a clean dataset. I pulled this from http://insideairbnb.com/get-the-data.html for London (02 June, 2019)
Download link: http://data.insideairbnb.com/united-kingdom/england/london/2019-06-05/data/listings.csv.gz

While loading the data in - I noticed that the prices in the data had money signs and were loaded in as strings. We need to take out "$" signs and convert their data types to a float since they have to do with money.

In [4]:
%%time
from pyspark.sql import functions as F

# tried to iterate through with this function but for some reason it would just regex the last column...
def cleanColumn(tmpdf,colName,findChar,replaceChar):
  """Pass in DF, column, and character to replace and replaces it with replaceChar variable"""
  tmpdf = tmpdf.withColumn(colName, F.regexp_replace(colName, findChar, replaceChar))
  return tmpdf

def fillNA(tmpdf, colName, nullReplacement):
  """pass DF, column name, and what to replace nulls with"""
  tmpdf = tmpdf.fillna({colName:nullReplacement})
  return tmpdf

def changeColumnDataType(tmpdf, colName, dataType):
  """pass in DF, column name, and datatype to convert to"""
  tmpdf = tmpdf.withColumn(colName, tmpdf[colName].cast(dataType))
  return tmpdf

monetaryColumns = ['price', 'weekly_price', 'monthly_price', 'security_deposit', 'cleaning_fee', 'extra_people']

# this is calling the function to try and iterate through and regex the columns I needed but would not work.
charToRemove='\W'
replaceWith =''
nullReplacement = '0'
dataType = "float"
for colName in monetaryColumns:
  listings = cleanColumn(listings,colName,charToRemove,replaceWith)
  listings = fillNA(listings, colName, nullReplacement)
  listings = changeColumnDataType(listings, colName, dataType)

# the columns we're interested in are 'price', 'weekly_price', 'monthly_price', 'security_deposit', 'cleaning_fee', 'extra_people'
# listings.select('price', 'weekly_price', 'monthly_price', 'security_deposit', 'cleaning_fee', 'extra_people').show()

# notice there are some null values. let's change those to 0.
  
listings.select('price', 'weekly_price', 'monthly_price', 'security_deposit', 'cleaning_fee', 'extra_people').show()

print(listings.select('price', 'weekly_price', 'monthly_price', 'security_deposit', 'cleaning_fee', 'extra_people').dtypes)

print(len(listings.columns))

+-------+------------+-------------+----------------+------------+------------+
 price|weekly_price|monthly_price|security_deposit|cleaning_fee|extra_people|
+-------+------------+-------------+----------------+------------+------------+
 8800.0| 64500.0| 235000.0| 35000.0| 5000.0| 2000.0|
 6500.0| 33300.0| 117600.0| 10000.0| 1500.0| 1500.0|
10000.0| 60000.0| 225000.0| 15000.0| 5000.0| 0.0|
30000.0| 137800.0| 0.0| 35000.0| 6500.0| 1000.0|
17500.0| 105000.0| 350000.0| 25000.0| 7000.0| 0.0|
 6500.0| 63000.0| 251500.0| 25000.0| 5000.0| 1100.0|
 2900.0| 19500.0| 79500.0| 10000.0| 0.0| 800.0|
14700.0| 72700.0| 274700.0| 10000.0| 7500.0| 1500.0|
 7500.0| 51000.0| 220000.0| 20000.0| 2500.0| 0.0|
 3400.0| 23100.0| 86800.0| 0.0| 1000.0| 1200.0|
10000.0| 0.0| 0.0| 0.0| 0.0| 0.0|
 4500.0| 0.0| 0.0| 0.0| 0.0| 2000.0|
19500.0| 0.0| 0.0| 0.0| 0.0| 0.0|
 7200.0| 0.0| 160000.0| 0.0| 0.0| 0.0|
 8000.0| 65000.0| 0.0| 0.0| 3000.0| 0.0|
 7000.0| 0.0| 0.0| 0.0| 0.0| 1000.0|
 6500.0| 0.0| 0.0| 0.0| 0.0| 0.0|
 7900.0| 53000.0| 210000.0| 0.0| 0.0| 0.0|
 6200.0| 0.0| 0.0| 0.0| 0.0| 0.0|
29600.0| 0.0| 0.0| 30000.0| 6000.0| 1800.0|
+-------+------------+-------------+----------------+------------+------------+
only showing top 20 rows

[('price', 'float'), ('weekly_price', 'float'), ('monthly_price', 'float'), ('security_deposit', 'float'), ('cleaning_fee', 'float'), ('extra_people', 'float')]
106
CPU times: user 29.1 ms, sys: 4.2 ms, total: 33.3 ms
Wall time: 1.7 s

In [5]:
# IGNORE THIS CELL. THIS IS BACKUP. 
# %%time
# from pyspark.sql import functions as F

# # tried to iterate through with this function but for some reason it would just regex the last column...
# # def cleanColumn(tmpdf,colName,findChar,replaceChar):
# #   """Pass in DF, column, and character to replace and replaces it with replaceChar variable"""
# #   tmpdf = tmpdf.withColumn(colName, F.regexp_replace(colName, findChar, replaceChar))
# #   return tmpdf

# def fillNA(tmpdf, colName, nullReplacement):
#   """pass DF, column name, and what to replace nulls with"""
#   tmpdf = tmpdf.fillna({colName:nullReplacement})
#   return tmpdf

# def changeColumnDataType(tmpdf, colName, dataType):
#   """pass in DF, column name, and datatype to convert to"""
#   tmpdf = tmpdf.withColumn(colName, tmpdf[colName].cast(dataType))
#   return tmpdf

# monetaryColumns = ['price', 'weekly_price', 'monthly_price', 'security_deposit', 'cleaning_fee', 'extra_people']

# # this is calling the function to try and iterate through and regex the columns I needed but would not work.
# # charToRemove='\W'
# # replaceWith =''
# # for colName in monetaryColumns:
# #   listings_clean = cleanColumn(listings,colName,charToRemove,replaceWith)

# # regex column values to take out dollar signs
# # honestly this was the most frustrating thing ever. I had to make 6 separate dataframes? I would try to overwrite the listings_clean dataframe but it would only regex the last column.
# listings_clean = listings.withColumn('price', F.regexp_replace('price', '\W', ''))
# listings_clean_2 = listings_clean.withColumn('weekly_price', F.regexp_replace('weekly_price', '\W', ''))
# listings_clean_3 = listings_clean_2.withColumn('monthly_price', F.regexp_replace('monthly_price', '\W', ''))
# listings_clean_4 = listings_clean_3.withColumn('security_deposit', F.regexp_replace('security_deposit', '\W', ''))
# listings_clean_5 = listings_clean_4.withColumn('cleaning_fee', F.regexp_replace('cleaning_fee', '\W', ''))
# listings_clean_6 = listings_clean_5.withColumn('extra_people', F.regexp_replace('extra_people', '\W', ''))

# # the columns we're interested in are 'price', 'weekly_price', 'monthly_price', 'security_deposit', 'cleaning_fee', 'extra_people'
# listings_clean_6.select('price', 'weekly_price', 'monthly_price', 'security_deposit', 'cleaning_fee', 'extra_people').show()

# # notice there are some null values. let's change those to 0.

# nullReplacement = '0'
# for col in monetaryColumns:
#   listings_clean_6 = fillNA(listings_clean_6, col, nullReplacement)
  
# listings_clean_6.select('price', 'weekly_price', 'monthly_price', 'security_deposit', 'cleaning_fee', 'extra_people').show()

# # now that nulls are cleaned up, change the datatype to float
# dataType = "float"
# for col in monetaryColumns:
#   listings_clean_6 = changeColumnDataType(listings_clean_6, col, dataType)

# print(listings_clean_6.select('price', 'weekly_price', 'monthly_price', 'security_deposit', 'cleaning_fee', 'extra_people').dtypes)

# print(len(listings_clean_6.columns))

In [6]:
%%time
display(listings.select("price"))

price
8800.0
6500.0
10000.0
30000.0
17500.0
6500.0
2900.0
14700.0
7500.0
3400.0


In [7]:
%%time
display(listings.select(F.log(F.col("price"))))

LOG(price)
9.082507000466299
8.779557455883728
9.210340371976184
10.308952660644293
9.769956159911606
8.779557455883728
7.9724660159745655
9.595602772766828
8.922658299524402
8.131530710604252


In [8]:
%%time
display(listings.groupBy("neighbourhood_cleansed").count())

neighbourhood_cleansed,count
Wandsworth,4135
Croydon,1081
Bexley,240
Lambeth,4712
Barking and Dagenham,334
Camden,5830
Greenwich,1640
Newham,1926
Tower Hamlets,7885
Barnet,1564


In [9]:
%%time
from pyspark.sql.functions import desc
display(listings.groupBy("neighbourhood_cleansed").count().sort(desc("count")))

neighbourhood_cleansed,count
Westminster,9275
Tower Hamlets,7885
Hackney,6007
Camden,5830
Kensington and Chelsea,5770
Islington,5020
Southwark,4784
Lambeth,4712
Wandsworth,4135
Hammersmith and Fulham,4134


In [10]:
%%time
avgPrice = listings.select(F.avg((listings['price']))).collect()[0]["avg(price)"]
print("Average price: ", avgPrice)

medianPrice = listings.approxQuantile('price', [0.5], 0.25)[0]
print("Median: ", medianPrice)

Average price: 11894.250813736606
Median: 7700.0
CPU times: user 90.9 ms, sys: 30.4 ms, total: 121 ms
Wall time: 27.8 s

In [11]:
%%time
predDF = listings.withColumn('avgPrediction', F.lit(avgPrice).cast("float"))
predDF = predDF.withColumn('medianPrediction', F.lit(medianPrice).cast("float"))

print(len(predDF.columns))

predDF.select('avgPrediction', 'medianPrediction').show()

108
+-------------+----------------+
avgPrediction|medianPrediction|
+-------------+----------------+
 11894.251| 7700.0|
 11894.251| 7700.0|
 11894.251| 7700.0|
 11894.251| 7700.0|
 11894.251| 7700.0|
 11894.251| 7700.0|
 11894.251| 7700.0|
 11894.251| 7700.0|
 11894.251| 7700.0|
 11894.251| 7700.0|
 11894.251| 7700.0|
 11894.251| 7700.0|
 11894.251| 7700.0|
 11894.251| 7700.0|
 11894.251| 7700.0|
 11894.251| 7700.0|
 11894.251| 7700.0|
 11894.251| 7700.0|
 11894.251| 7700.0|
 11894.251| 7700.0|
+-------------+----------------+
only showing top 20 rows

CPU times: user 9.81 ms, sys: 0 ns, total: 9.81 ms
Wall time: 518 ms

In [12]:
%%time
from pyspark.ml.evaluation import RegressionEvaluator

regressionMeanEvaluator = RegressionEvaluator(predictionCol="avgPrediction", labelCol="price", metricName="rmse")

print("The RMSE for predicting the average price is: ", regressionMeanEvaluator.evaluate(predDF))

regressionMedianEvaluator = RegressionEvaluator(predictionCol="medianPrediction", labelCol="price", metricName="rmse")
  
print("The RMSE for predicting the median price is: ", regressionMedianEvaluator.evaluate(predDF))

The RMSE for predicting the average price is: 21437.043227257076
The RMSE for predicting the median price is: 21843.501601524466
CPU times: user 93.9 ms, sys: 40.2 ms, total: 134 ms
Wall time: 32.4 s

This is like the worst RMSE ever since it's just an average price and median. The compute time is what we're interested in. To calculate RMSE you have to compute each row with this formula: